# Miller Predicting a Continuous Target with Regression

**Author:** Dan Miller

**Date:** November 10th, 2025

**Objective:** Be able to predict a continuous numeric target, fare, using features in the Titanic dataset

## Introduction

This project explores the seaborn Titanic dataset.  With the goal of being able to predict a continuous target with regression, linear regression as well as some alternative models will all be created and evaluated in terms of predicting fare, the price of the journey.  The data will be explored and prepped before the models are created.  There will be a summary at the end to discuss the findings.

## Imports

In [4]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import ElasticNet, LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures


## Section 1. Import and Inspect the Data

In [5]:
# Load the Titanic dataset
titanic = sns.load_dataset("titanic")

# Inspect the first few rows of the dataset
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Section 2. Data Exploration and Preparation

In [6]:
# Impute missing values for age using median
titanic["age"] = titanic["age"].fillna(titanic["age"].median())

# Drop rows with missing fare
titanic = titanic.dropna(subset=["fare"])

# Cfreate a new feature for family size
titanic["family_size"] = titanic["sibsp"] + titanic["parch"] + 1

# Map categorical variable embarked to numerical values
titanic["embarked"] = titanic["embarked"].map({"C": 0, "Q": 1, "S": 2})